In [1]:
import random
import numpy as np


In [2]:
class Pop:
   def __init__(self, tamanho_tabuleiro, quanti_rainhas):
      self.tamanho_tabuleiro = tamanho_tabuleiro
      self.qunti_rainhas = quanti_rainhas
   def gera_pop_inicial(self):
      pop_inicial = []
      while len(pop_inicial) < self.qunti_rainhas:
         xy = []
         xy.append(random.randint(0, self.tamanho_tabuleiro -1))
         xy.append(random.randint(0, self.tamanho_tabuleiro -1))
         if xy in pop_inicial:
            print("posição já existente!")
         else:
            pop_inicial.append(xy)
      return pop_inicial
   

In [3]:
class Custos:

    def __init__(self, lista):
       self.lista = lista
       #self.lista_temp = []

    def calculo_custo_diag(self):
      cont = 0
      lista_verificada = []
      
      
      
      def calculo_custo_LC():
         lista_x = []
         lista_y = []
         cont = 0
         for i in range(len(self.lista)):
            pop = self.lista[i]
            lista_x.append(pop[0])
            lista_y.append(pop[1])
         for j in range(0,len(lista_x)):
            soma_x = lista_x.count(j)
            soma_y = lista_y.count(j)
            if soma_x == 0 or soma_x == 1:
               soma_x = 0
            if soma_y == 0 or soma_y == 1:
               soma_y = 0
            if soma_x > 0:
               soma_x = soma_x-1
            if soma_y > 0:
               soma_y = soma_y-1
            cont = cont+(soma_x+soma_y) 
         return cont
          
      custoLC = calculo_custo_LC()
      
      def verifica_lc(lista1, lista2):
        verifica = False
        if lista1[0] == lista2[0] or lista1[1] == lista2[1]:
           verifica = True
        return verifica
      
      for i in range(0, len(self.lista)):
         lista_temp =  self.lista[:]        
         posi = self.lista[i]
         lista_temp.pop(i)
         for j in range(0, len(lista_temp)):
            lista1 = []
            
            verifica = lista_temp[j]
            
            lista1.append(verifica)
            lista1.append(posi)
            
            verfica_colunas = verifica_lc(posi, verifica)
            
            if lista1 in lista_verificada or verfica_colunas == True:
                print("já foi verificado / ta na mesma linha ou coluna!")

            elif ( posi[0] + posi[1]) == (verifica[0] + verifica[1])  or (posi[0] - posi[1]) == (verifica[0] - verifica[1]) :
                lista2 = []
                lista2.append(posi)
                lista2.append(verifica)
                lista_verificada.append(lista2)              
                cont+=1       
      return  custoLC, cont,custoLC+cont, lista_verificada 

In [4]:
teste = Pop(8,8)
teste2 = teste.gera_pop_inicial()
custo = Custos(teste2)

print(custo.calculo_custo_diag())
#print(custo.calculo_custo_LC())



posição já existente!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
já foi verificado / ta na mesma linha ou coluna!
(5, 4, 9, [[[4, 1], [7, 4]], [[2, 7], [5, 4]], [[4, 6], [3, 5]], [[4, 7], [7, 4]]])
